# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [3]:
# headers={
#     'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
# }
# def get_content(url):
#     with requests.Session() as req:
#         req.headers.update(headers)
#         r = req.get(url).content
#     return r

# url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = get_content(url)
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
df=pd.read_csv("data-20160608T1215-structure-20160608T1215.csv", sep=',', index_col=0)
display(df)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [11]:
# нужно распарсить данные

# будем добавлять "норма" или "нарушение"
results = []

for idx, row in df.iterrows():
  s = row['Норматив']
  analysis = 0
  try: 
    analysis = float(row['Результат анализа'])
  except:
    analysis = 0
  # выбираем варианты
  if re.search('не\sболее', s):
    attempt = re.search('(\d{1,5},?\d{0,5})-(\d{1,5},?\d{0,5})', s)
    if attempt:
      a = float(attempt.group(1).replace(',', '.'))
      b = float(attempt.group(2).replace(',', '.'))
      bound = max(a, b)
      if analysis <= bound:
        results.append('норма')
      else:
        results.append('нарушение')
    else:
      num = re.search('(\d{1,5},?\d{0,5})', s).group().replace(',', '.')
      bound = float(num)
      if analysis <= bound:
        results.append('норма')
      else:
        results.append('нарушение')
  elif re.search('в\sпределах', s):
    a = float(re.search('(\d{1,5},?\d{0,5})-(\d{1,5},?\d{0,5})', s).group(1).replace(',', '.'))
    b = float(re.search('(\d{1,5},?\d{0,5})-(\d{1,5},?\d{0,5})', s).group(2).replace(',', '.'))
    if a <= analysis <= b:
      results.append('норма')
    else:
      results.append('нарушение')
  else:
    raise Exception("Unknown Data Format")

df['Результат_по_норме'] = results

In [12]:
# итоговый датафрейм
df

,Единица измерений,Результат анализа,Норматив,Результат_по_норме
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,норма
Запах,баллы,1,не более 2-3,норма
Цветность,градусы,б/цвета,не более 30,норма
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",норма
Нитраты (по NO3),мг/дм3,24,не более 45,норма
Фосфаты (P),мг/дм3,0.36,"не более 3,5",норма
Хлориды (Cl),мг/дм3,200,не более 350,норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
import random

# пусть 0 - апельсин, 1 - яблоко
# выбирает 3 фрукта из ящика случайно
def make_experiment():
  apples = [1 for i in range(4)]
  oranges = [0 for i in range(5)]
  fruits = apples + oranges
  result = []

  for i in range(3):
    elem = random.choice(fruits)
    result.append(elem)
    fruits.remove(elem)
  stat = sum(result)
  # если сумма равна нулю - выбрали три апельсина
  if stat == 0:
    return 1
  else:
    return 0

# итоговая вероятность - сумма успехов на все события
# N - размер выборки
def run(N):
  stats =[]
  for i in range(N):
    stats.append(make_experiment())
  p = sum(stats)/len(stats)
  print(f'Final probability for sample len {N}: {p:.3f}')

run(10)
run(100)
run(1000)
run(10000)
run(100000)
run(1000000)

Final probability for sample len 10: 0.100
Final probability for sample len 100: 0.180
Final probability for sample len 1000: 0.112
Final probability for sample len 10000: 0.121
Final probability for sample len 100000: 0.117
Final probability for sample len 1000000: 0.119


Ответ: да, с увеличением размера выборки получаем нужную вероятность. Несмотря на то, что генераторы случайных чисел **псевдослучайные** и основаны на линейных рекурентных последовательностях, которые не гарантируют абсолютную случайность. (лишь период $2^{n}-1$ в лучшем случае).

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# 1 - стандартная, 0 - нестандартная
appropriate = [1 for i in range(7)]
not_appropr = [0 for i in range(3)]
details = appropriate + not_appropr

def make_experiment():
  num_checks = 0
  success_attempts = 0

  # пока не выбрали стандартную
  while success_attempts < 1:
    # если проверили больше двух деталей - нет смысла проверять дальше
    if num_checks >= 2:
      return 0
    elem = random.choice(details)
    num_checks += 1
    if elem == 1:
      success_attempts += 1

  # условие задачи
  if num_checks == 2:
    return 1
  else:
    return 0

run(10)
run(100)
run(1000)
run(10000)
run(100000)
run(1000000)
run(10000000)

Final probability for sample len 10: 0.200
Final probability for sample len 100: 0.270
Final probability for sample len 1000: 0.213
Final probability for sample len 10000: 0.213
Final probability for sample len 100000: 0.212
Final probability for sample len 1000000: 0.210
Final probability for sample len 10000000: 0.210


Тут результаты похуже, чем в прошлой задаче. Разница целых 0.23 от теоретической вероятности. Т.е. для этой задачи результат не подтверждается эксперементально